Модуль для тестирования в Django работает по тому же принципу, что и библиотека unittest: 

* разработчик создаёт классы, унаследованные от базового класса TestCase;

* в этих классах описываются методы, названия которых должны начинаться с префикса `test_`;

* каждый такой метод — это отдельный тест.

>Логически связанные между собой тесты объединяют в классы, которые наследуются от базового класса `TestCase`.

Проверка утверждений, как и в unittest, проводится с помощью специальных методов класса **TestCase**; названия этих методов начинаются со слова `assert`: `assertEqual`, `assertGreater`, `assertFalse`… 

***
## Первые тесты

При работе с тестами первым делом надо импортировать класс TestCase из пакета `django.test`: все классы тестов должны наследоваться от него. Сам он, в свою очередь, наследуется от класса `unittest.TestCase`, но не напрямую, а через цепочку промежуточных классов.



In [ ]:
# news/tests/test_trial.py
from django.test import TestCase


class Test(TestCase):

    def test_example_success(self):
        self.assertTrue(True)  # Этот тест всегда будет проходить успешно.


class YetAnotherTest(TestCase):

    def test_example_fails(self):
        self.assertTrue(False)  # Этот тест всегда будет проваливаться. 


Здесь два класса с тестами; эти тесты проверяют не код проекта, а простые утверждения:

* *True — это True?* Тест всегда будет успешным.

* *False — это True?* Тест всегда будет проваливаться.

Эти тесты послужат для демонстрации некоторых возможностей библиотеки `django.test`.

***
## Выборочный запуск тестов

Запустите виртуальное окружение проекта; при тестировании проекта через модуль `django.test` запускать сервер разработчика не нужно: модуль сам выполнит все необходимые операции.


In [ ]:
# Запустить все тесты проекта.
python manage.py test 

В консоли должно появиться сообщение о результатах:


Отчёт о выполнении:

* `.F` — один тест прошёл (точка), другой провалился (`F`)

* `FAIL: test_example_fails …` — какой конкретно тест провален

* `AssertionError: False is not true` — причина провала теста

* `Ran 2 tests in 0.003s` — общая сводка по выполненным тестам

Если вдруг тесты не запустились и в отчёте вы видите `Ran 0 tests in 0.000s` — проверьте, что в директории с тестами есть файл `__init__.py`: без него модуль тестирования Django не станет искать файлы с тестами в этой директории.

Иногда нет необходимости выполнять все тесты проекта, а нужно запустить лишь один из них или определённую группу. Как и в библиотеке unittest, для выборочного запуска тестов можно указать путь к нужному пакету, модулю, тестирующему классу и методу:


In [ ]:

# Запустить все тесты проекта.
python manage.py test

# Запустить только тесты в приложении news.
python manage.py test news

# Запустить только тесты из файла test_trial.py в приложении news.
python manage.py test news.tests.test_trial

# Запустить только тесты из класса Test
# в файле test_trial.py приложения news.  
python manage.py test news.tests.test_trial.Test

# Запустить только тест test_example_fails
# из класса YetAnotherTest в файле test_trial.py приложения news.
python manage.py test news.tests.test_trial.YetAnotherTest.test_example_fails 

***
## Больше информации о результатах теста

Развёрнутый отчёт о результатах теста можно получить, выполнив команду `python manage.py test` с параметром `--verbosity` (или `-v`); значениями этого параметра могут быть числа от 0 до 3: чем больше значение — тем подробнее отчёт.

Если этот параметр не указан явно — по умолчанию он устанавливается равным единице:



In [ ]:
python manage.py test
# Это то же самое, что 
python manage.py test -v 1 


Чтобы увидеть развёрнутый список пройденных и проваленных тестов — запустите тесты с параметром `-v 2`,



In [ ]:
python manage.py test -v 2 

Помимо списка тестов будет выведена и дополнительная информация о каждом тесте.

Проведите тесты с различными значениями `verbosity` от 0 до 3 и посмотрите, чем отличается вывод результатов.

***
## Тестовая база данных

При выполнении тестов в консоли появляется информация о базе данных, которая создаётся при старте тестов и удаляется после их окончания:

![alt text](https://pictures.s3.yandex.net/resources/image_1700244714.png)

При запуске с параметром `-v 2` или `-v 3` видно больше подробностей.

Сначала создаётся база данных, затем к ней применяются миграции — и только после этого выполняются тесты. 

Дело в том, что при тестировании проекта необходимо взаимодействовать с базой данных — например, создавать объекты моделей или пользователей. Перед тестированием в оперативной памяти **создаётся временная база данных**; её структура полностью копирует структуру существующей базы Django-проекта (именно для этого и выполняются миграции), однако эта база пуста, в ней нет ни объектов, ни пользователей — ничего. 

Все данные в этой базе нужно создавать в процессе тестирования или на этапе подготовки к тестам. Запросы при тестировании делаются именно к временной базе, основная база не затрагивается.

По окончании тестирования тестовая база автоматически удаляется. Такая схема работы позволяет не замусоривать реальную базу данных тестовыми записями. 

Создание и удаление временной базы в оперативной памяти выполняются при настройках проекта по умолчанию, однако эти настройки могут быть изменены.

***
## Создание объектов для тестов

В unittest объекты для тестов создаются при помощи **фикстур** — специальных методов класса TestCase. В этих методах подготавливаются данные для тестирования:

* метод `setUp()` выполняется перед каждым тестом;

* метод `setUpClass()` выполняется только один раз, перед выполнением всех тестов класса.

Каждый тест должен быть независим от других, и если в нескольких тестах используется один и тот же объект — было бы логично создавать его методом `setUp()` —  перед каждым тестом заново.

Однако заново создавать объекты для каждого теста нет необходимости. Дело в том, что при работе с модулем `django.test` все тесты класса `django.test.TestCase` проводятся в **транзакциях базы данных**. Это означает, что все изменения базы данных, выполненные в ходе проведения теста, после завершения этого теста «откатываются» назад, к исходному состоянию — удалённые объекты восстанавливаются, новые объекты удаляются; восстанавливаются и все значения всех полей.

Классы с тестами тоже независимы друг от друга: каждый тестовый класс тоже «обёрнут» в транзакцию. Объекты, созданные в классе, удаляются после выполнения тестов класса. Следующий тестовый класс будет работать с базой данных, в которой нет объектов, созданных другими классами. 

Работа в транзакциях заметно упрощает написание тестов, и это еще один аргумент для того, чтобы использовать для тестирования готовые библиотеки, а не выдумывать что-то своё.

***
## Метод setUpTestData() вместо setUpClass()

После выполнения теста модуль `django.test` восстанавливает исходное состояние данных, созданных для тестирования. Значит, при выполнении тестов в классе необязательно готовить данные перед каждым тестом, а достаточно подготовить данные один раз: например, это можно было бы сделать в методе `setUpClass()`. Однако в тестирующем модуле Django этот метод работает не так, как в обычном unittest.

Если вызвать метод `setUpClass()` в классе, унаследованном от `django.test.TestCase` — программа выбросит ошибку:


In [ ]:

AttributeError: type object '<имя_класса>' has no attribute 'cls_atomics' 

Эта ошибка возникает именно в тестирующем модуле Django. Есть способ избежать её: при вызове метода `setUpClass()` первым делом нужно обратиться к родительскому методу `setUpClass()` через функцию `super()` — и только после этого выполнять остальной код этого метода:


In [ ]:
from django.test import TestCase


class DemonstrationExample(TestCase):

    @classmethod
    def setUpClass(cls):        
        super().setUpClass()  # Вызов метода setUpClass() из родительского класса.
        # А здесь код, который подготавливает данные
        # перед выполнением тестов этого класса. 

То же самое касается и метода `tearDownClass()`: [документация предписывает](https://docs.djangoproject.com/en/5.1/topics/testing/tools/#django.test.TestCase.setUpTestData) и для него вызывать `super().tearDownClass()`, хотя даже и без вызова родительского метода ошибки не возникнет. 

Для метода `tearDownClass()` родительский метод вызывается **после** операций, описанных в этом методе: сначала «подчистили» за собой  — и после этого вызвали родительский `tearDownClass()`:

In [ ]:
from django.test import TestCase


class MyTestCase(TestCase):

    @classmethod
    def setUpClass(cls):
        super().setUpClass()
        ...

    @classmethod
    def tearDownClass(cls):
        ...  # Выполнение необходимых операций.
        super().tearDownClass()  # Вызов родительского метода. 

В  классе `django.test.TestCase` есть более удобный способ для создания тестовых объектов — это метод `setUpTestData()`. Он похож на метод `setUpClass()`, но не требует явного вызова родительского метода. При тестировании Django через модуль `django.test` рекомендуется работать именно с методом `setUpTestData()`.

>Обязательный вызов `super()` в `setUpClass()` и применение метода `setUpTestData()` — это особенности пакета `django.test`.

>В Python-библиотеке unittest нет метода `setUpTestData()`, а в методе `setUpClass()` не нужно вызывать `super()`

In [ ]:
# news/tests/test_trial.py
from django.test import TestCase

# Импортируем модель, чтобы работать с ней в тестах.
from news.models import News


# Создаём тестовый класс с произвольным названием, наследуем его от TestCase.
class TestNews(TestCase):

    # В методе класса setUpTestData создаём тестовые объекты.
    # Оборачиваем метод соответствующим декоратором.    
    @classmethod
    def setUpTestData(cls):
        # Стандартным методом Django ORM create() создаём объект класса.
        # Присваиваем объект атрибуту класса: назовём его news.
        cls.news = News.objects.create(
            title='Заголовок новости',
            text='Тестовый текст',
        )

    # Проверим, что объект действительно был создан.
    def test_successful_creation(self):
        # При помощи обычного ORM-метода посчитаем количество записей в базе.
        news_count = News.objects.count()
        # Сравним полученное число с единицей.
        self.assertEqual(news_count, 1)

Теперь в любом тесте можно обращаться к экземпляру класса `News` при помощи конструкции `self.news`, где `self` ссылается на сам объект; к этому же объекту можно обратиться и через атрибут класса: `TestNews.news`. 

Если в тестах не был создан **атрибут объекта** с названием `news` (одноимённый с **атрибутом класса**) — не будет никакой ошибки в том, чтобы обращаться к объекту через `self.news`. 

Обратимся к объекту и проверим, что у новости именно такой заголовок, как ожидается. Добавьте в класс `TestNews` ещё один метод:

In [ ]:
# news/tests/test_trial.py
...
class TestNews(TestCase): 

    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(
            title='Заголовок новости',
            text='Тестовый текст',
        )

    def test_successful_creation(self):
        news_count = News.objects.count()
        self.assertEqual(news_count, 1)

    def test_title(self):
        # Сравним свойство объекта и ожидаемое значение.
        self.assertEqual(self.news.title, 'Заголовок новости')   

Оба теста выполнены на «отлично», всё работает, но в тестах нарушен принцип DRY: текст заголовка повторяется в двух местах кода. При работе со строками лучше сохранять их в константы класса:

In [ ]:
# news/tests/test_trial.py
from django.test import TestCase

from news.models import News


class TestNews(TestCase):
    # Все нужные переменные сохраняем в атрибуты класса.
    TITLE = 'Заголовок новости'
    TEXT = 'Тестовый текст'
    
    @classmethod
    def setUpTestData(cls):
        cls.news = News.objects.create(
            # При создании объекта обращаемся к константам класса через cls.
            title=cls.TITLE,
            text=cls.TEXT,
        )

    def test_successful_creation(self):
        news_count = News.objects.count()
        self.assertEqual(news_count, 1)

    def test_title(self):
        # Чтобы проверить равенство с константой -
        # обращаемся к ней через self, а не через cls:
        self.assertEqual(self.news.title, self.TITLE) 

***
## Программный HTTP-клиент

Django — это веб-фреймворк, и при тестировании важно проверить, что данные на страницах отображаются как надо, что формы работают правильно, а пользователи получают доступ к страницам в соответствии с их правами. Для проверки всех этих случаев будет логично обратиться к страницам проекта и к URL, которые обрабатывает фреймворк. Для этого  используется специальный программный HTTP-клиент, имитирующий работу браузера.

Этот программный клиент может:

* имитировать GET- и POST-запросы, проверять заголовки ответов и содержимое страниц;

* работать от имени авторизованного или неавторизованного пользователя;

* отслеживать редиректы, проверять URL и код статуса при каждом редиректе;

* проверять, какие HTML-шаблоны применяются для рендера запрошенной страницы и что передаётся в словаре `context`;

Для создания программного клиента в модуле `django.test` есть класс `Client()`. Каждый экземпляр этого класса — это отдельный веб-клиент, которым можно управлять из кода. 

В каждом тестирующем классе по умолчанию создаётся объект веб-клиента; доступ к нему можно получить через атрибут `self.client`. Для каждого теста Django создаёт новый объект клиента, поэтому запросы клиента в определённом тесте никак не влияют на другие тесты; в части работы с клиентами и запросами тесты будут полностью независимы.

При тестировании можно создать несколько клиентов: в одном можно авторизоваться, а из другого клиента работать без авторизации, тестируя сценарии для анонимных пользователей. При этом в каждом тестирующем классе будет доступен и клиент, созданный автоматически.

In [ ]:
# Импортируем функцию для определения модели пользователя.
from django.contrib.auth import get_user_model
from django.test import Client, TestCase

# Получаем модель пользователя.
User = get_user_model()


class TestNews(TestCase):

    @classmethod
    def setUpTestData(cls):
        # Создаём пользователя.
        cls.user = User.objects.create(username='testUser')
        # Создаём объект клиента.
        cls.user_client = Client()
        # "Логинимся" в клиенте при помощи метода force_login().        
        cls.user_client.force_login(cls.user)
        # Теперь через этот клиент можно отправлять запросы
        # от имени пользователя с логином "testUser".

Если в тестовом классе неважно, какой именно username будет у пользователя, и username используется только в одном месте кода — можно не выносить его в константу класса, как это было сделано в предыдущем примере — с заголовком и текстом новости.

***
## Объект response: ответ на запрос

В ответ на любой запрос, отправленный через клиент, возвращается специальный [объект класса Response](https://docs.djangoproject.com/en/5.1/topics/testing/tools/#django.test.Response). В нём содержится ответ сервера и дополнительная информация. Стоит обратить внимание на следующие атрибуты:

* `response.status_code` — содержит код ответа запрошенного адреса;

* `response.content` — данные ответа в виде строки байтов;

* `response.context` — словарь переменных, переданный для отрисовки шаблона при вызове функции `render()`;

* `response.templates` — перечень шаблонов, вызванных для отрисовки запрошенной страницы;

Посмотрим, как всё это работает на практике. Для тестирования путей в вашем проекте подготовлен файл *news/tests/test_routes.py*, так что работу продолжим именно в нём.

Добавьте в файл *news/tests/test_routes.py* код из листинга и запустите тесты:

In [ ]:
# news/tests/test_routes.py
from django.test import TestCase


class TestRoutes(TestCase):

    def test_home_page(self):
        # Вызываем метод get для клиента (self.client)
        # и загружаем главную страницу.
        response = self.client.get('/')
        # Проверяем, что код ответа равен 200.
        self.assertEqual(response.status_code, 200) 

Отличный результат: главная страница проекта отвечает кодом 200, проект работает — и это уже немало.

Однако код теста стоит немного улучшить:

* вместо числа 200 применим константы из встроенной библиотеки http,

* вместо относительного адреса главной страницы используем `name` пути в функции `reverse()`. Сохраним её в переменную `url`. Вызов `reverse('news:home')` вернёт строку `'/'`.

Импортируйте класс `http.HTTPStatus` и функцию `reverse()` из `django.urls`; измените код в классе `TestRoutes`:

In [ ]:
# news/tests/test_routes.py
# Импортируем класс HTTPStatus.
from http import HTTPStatus

from django.test import TestCase
# Импортируем функцию reverse().
from django.urls import reverse


class TestRoutes(TestCase):

    def test_home_page(self):
        # Вместо прямого указания адреса 
        # получаем его при помощи функции reverse().
        url = reverse('news:home')
        response = self.client.get(url)
        # Проверяем, что код ответа равен статусу OK (он же 200).
        self.assertEqual(response.status_code, HTTPStatus.OK) 